# Customer Churn Prediction with Ray on Snowflake

This notebook demonstrates distributed ML model training using Ray on Snowflake.

**What we'll do:**
1. Connect to Ray cluster and scale it dynamically
2. Load customer data from Interactive Tables
3. Engineer features for churn prediction
4. Train XGBoost model using Ray distributed training
5. Evaluate model performance
6. Save model to Snowflake Model Registry

**Prerequisites:**
- Run this in a Snowflake Notebook with ML Runtime
- Data should exist in AUTOMATED_INTELLIGENCE.INTERACTIVE.CUSTOMER_ORDER_ANALYTICS

## 1. Setup: Connect to Ray Cluster

In [ ]:
import ray
from snowflake.ml.runtime_cluster import scale_cluster, get_nodes, get_ray_dashboard_url
import snowflake.snowpark as snowpark
from snowflake.snowpark import functions as F
from snowflake.snowpark import Session
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import xgboost as xgb
from snowflake.ml.registry import Registry
import datetime
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# Get Snowflake session (automatically available in Snowflake Notebooks)
session = snowpark.context.get_active_session()

# Connect to Ray cluster
ray.init(address="auto", ignore_reinit_error=True)
print(f"Ray cluster resources: {ray.cluster_resources()}")
print(f"Current cluster size: {len(get_nodes())} nodes")

# Get Ray Dashboard URL
dashboard_url = get_ray_dashboard_url()
print(f"\n🎯 Ray Dashboard: {dashboard_url}")
print("   Open this URL in a new tab to monitor your cluster")

## 2. Scale Ray Cluster

For this demo, we'll scale to 4 nodes (1 head + 3 workers) to demonstrate distributed training.

In [ ]:
# Scale cluster to 4 nodes
print("Scaling cluster to 4 nodes...")
scale_cluster(expected_cluster_size=4)
print(f"✅ Cluster scaled to {len(get_nodes())} nodes")
print(f"Updated resources: {ray.cluster_resources()}")

## 3. Load Data from Snowflake

We'll load customer order analytics data and engineer features for churn prediction.

In [ ]:
# Load customer analytics data
df = session.table("AUTOMATED_INTELLIGENCE.INTERACTIVE.CUSTOMER_ORDER_ANALYTICS")

# Add derived columns
df = df.with_column('CUSTOMER_TENURE_DAYS',
    F.datediff('day', F.col('FIRST_ORDER_DATE'), F.col('LAST_ORDER_DATE')))

df = df.with_column('DAYS_SINCE_LAST_ORDER',
    F.datediff('day', F.col('LAST_ORDER_DATE'), F.current_date()))

# Filter to customers with orders
df = df.filter(F.col('TOTAL_ORDERS') > 0)

print(f"\n📊 Loaded {df.count():,} customers")
print("\nSample data:")
df.select('CUSTOMER_ID', 'TOTAL_ORDERS', 'TOTAL_SPENT', 'CUSTOMER_TENURE_DAYS', 'DAYS_SINCE_LAST_ORDER').show(5)

## 4. Feature Engineering

Create features for churn prediction:
- **Churn label**: Customer hasn't ordered in 7+ days (target variable)
- **Frequency**: Total orders (historical behavior)
- **Monetary**: Total spent, average order value (spending patterns)
- **Tenure**: Days as customer (customer age)
- **Order frequency**: Orders per day of tenure (engagement rate)

**Important:** We exclude `days_since_last_order` to avoid data leakage - it directly reveals the target variable.

**Note:** Using 7-day threshold for demo purposes to ensure balanced classes.

In [ ]:
# Define churn: customers who haven't ordered in 7+ days
CHURN_THRESHOLD_DAYS = 7

# Create churn label column
df = df.with_column('IS_CHURNED', 
    (F.col('DAYS_SINCE_LAST_ORDER') > CHURN_THRESHOLD_DAYS).cast('int'))

# Additional derived features (excluding leaky features)
df = df.with_column('ORDER_FREQUENCY', 
    F.col('TOTAL_ORDERS') / (F.col('CUSTOMER_TENURE_DAYS') + 1))

df = df.with_column('REVENUE_PER_ORDER', 
    F.col('TOTAL_SPENT') / F.col('TOTAL_ORDERS'))

# Select features for training (NO days_since_last_order - that's data leakage!)
feature_columns = [
    'TOTAL_ORDERS',
    'TOTAL_SPENT', 
    'AVG_ORDER_VALUE',
    'CUSTOMER_TENURE_DAYS',
    'ORDER_FREQUENCY',
    'REVENUE_PER_ORDER'
]

label_column = 'IS_CHURNED'

# Show dataset statistics
total_count = df.count()
churn_stats = df.group_by('IS_CHURNED').count().collect()

print(f"📊 Dataset Statistics:")
print(f"   Total samples: {total_count:,}")
print(f"\n   Class distribution:")
for row in churn_stats:
    status = "Active" if row['IS_CHURNED'] == 0 else "Churned"
    count = row['COUNT']
    pct = (count / total_count) * 100
    print(f"   {status} customers: {count:,} ({pct:.1f}%)")

# Calculate class imbalance ratio
active_count = [r['COUNT'] for r in churn_stats if r['IS_CHURNED'] == 0][0]
churned_count = [r['COUNT'] for r in churn_stats if r['IS_CHURNED'] == 1][0]
class_ratio = active_count / churned_count
print(f"\n   Class imbalance ratio: {class_ratio:.1f}:1")

## 5. Train Model with Ray

We'll use Ray Data for distributed data processing and XGBoost for training.

In [ ]:
# Convert to pandas for sklearn
pdf = df.select(feature_columns + [label_column]).to_pandas()

# Split features and target
X = pdf[feature_columns]
y = pdf[label_column]

# Train/test split (stratified to maintain class balance)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\n📊 Data Split:")
print(f"   Training set: {len(X_train):,} samples")
print(f"   Test set: {len(X_test):,} samples")

# Create XGBoost classifier with class imbalance handling
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    random_state=42,
    scale_pos_weight=class_ratio,  # Handle class imbalance
    tree_method='hist',  # Efficient distributed training
    n_jobs=-1  # Use all available cores
)

# Create pipeline with StandardScaler + XGBoost
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', xgb_model)
])

# Train pipeline (Ray handles distributed execution across cluster)
# Monitor Ray cluster during training
print("\n📊 Ray Cluster Status BEFORE Training:")
print(f"   Available CPUs: {ray.available_resources().get('CPU', 0)}")
print(f"   Available Memory: {ray.available_resources().get('memory', 0) / 1e9:.2f} GB")

print("\n⏳ Training XGBoost model with Ray distributed computing...")
print(f"   Using {len(get_nodes())} nodes in Ray cluster")
print(f"   Class imbalance ratio: {class_ratio:.1f}:1")

pipeline.fit(X_train, y_train)

print("✅ Training complete!")

# Check resource usage after training
print("\n📊 Ray Cluster Status AFTER Training:")
print(f"   Available CPUs: {ray.available_resources().get('CPU', 0)}")
print(f"   Available Memory: {ray.available_resources().get('memory', 0) / 1e9:.2f} GB")
print(f"   Model: Pipeline(StandardScaler + XGBClassifier)")
print(f"   Features: {len(feature_columns)}")
print(f"   Training samples: {len(X_train):,}")

## 6. Evaluate Model Performance

In [ ]:
# Make predictions using pipeline
y_pred = pipeline.predict(X_test)
y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

# Calculate metrics
roc_auc = roc_auc_score(y_test, y_pred_proba)

print("📊 Model Performance:")
print(f"\nROC-AUC Score: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Active', 'Churned']))

print("\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"                  Predicted")
print(f"                Active  Churned")
print(f"Actual Active    {cm[0,0]:6d}  {cm[0,1]:6d}")
print(f"       Churned   {cm[1,0]:6d}  {cm[1,1]:6d}")

# Calculate recall for churned customers
churned_recall = cm[1,1] / (cm[1,0] + cm[1,1])
churned_caught = cm[1,1]
churned_total = cm[1,0] + cm[1,1]
print(f"\n🎯 Business Impact:")
print(f"   Churned customers caught: {churned_caught}/{churned_total} ({churned_recall*100:.1f}%)")
print(f"   Missed churned customers: {cm[1,0]} ({(1-churned_recall)*100:.1f}%)")

## 7. Feature Importance

In [ ]:
# Get feature importance from the XGBoost model in the pipeline
xgb_model = pipeline.named_steps['classifier']
feature_importance = xgb_model.feature_importances_

# Create DataFrame for visualization
feature_importance_df = pd.DataFrame({
    'feature': feature_columns,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

print("\n🔍 Feature Importance:")
print(feature_importance_df.to_string(index=False))

print("\n💡 Insights:")
top_feature = feature_importance_df.iloc[0]['feature']
top_importance = feature_importance_df.iloc[0]['importance']
print(f"   Most important feature: {top_feature} ({top_importance:.2%})")
print(f"   This feature has the strongest predictive power for customer churn")

## 8. Save Model to Snowflake Model Registry

In [ ]:
# Create registry
registry = Registry(session=session, database_name="AUTOMATED_INTELLIGENCE", schema_name="MODELS")

# Prepare model metadata
model_name = "customer_churn_predictor"
version_name = f"v_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"

# Sample input for schema inference (UNSCALED data)
sample_input = X_test.head(5)

# Save pipeline (already trained above)
print(f"\n💾 Saving model to registry...")
print(f"   Model: {model_name}")
print(f"   Version: {version_name}")

mv = registry.log_model(
    model=pipeline,
    model_name=model_name,
    version_name=version_name,
    sample_input_data=sample_input,
    comment=f"XGBoost churn predictor (Ray) with preprocessing pipeline. Trained on {len(X_train):,} customers. ROC-AUC: {roc_auc:.4f}",
    metrics={
        "roc_auc": float(roc_auc),
        "training_samples": int(len(X_train)),
        "test_samples": int(len(X_test)),
        "churned_recall": float(churned_recall),
        "churn_rate": float(y.mean())
    }
)

print(f"✅ Model saved successfully!")
print(f"\n📦 Model Details:")
print(f"   Database: AUTOMATED_INTELLIGENCE")
print(f"   Schema: MODELS")
print(f"   Model: {model_name}")
print(f"   Version: {version_name}")
print(f"\n🎯 Next Steps:")
print(f"   1. View model in Snowsight: Data > Models > {model_name}")
print(f"   2. Check the Streamlit dashboard ML Insights page")
print(f"   3. Use model for predictions on new data")

## 9. Test Model Predictions

In [ ]:
# Get some sample customers to predict
sample_customers_df = df.limit(10)

sample_customers_df = df.limit(10)

# Convert to pandas using the SAME feature_columns list
sample_pdf_full = sample_customers_df.select(
    feature_columns + [label_column, 'CUSTOMER_ID', 'DAYS_SINCE_LAST_ORDER']
).to_pandas()

# Extract just the features in the exact same way as X_train was created
X_sample = sample_pdf_full[feature_columns]

# Use the pipeline for predictions
predictions = pipeline.predict_proba(X_sample)[:, 1]

# Display results
results = pd.DataFrame({
    'customer_id': sample_pdf_full['CUSTOMER_ID'],
    'total_orders': sample_pdf_full['TOTAL_ORDERS'],
    'days_since_last_order': sample_pdf_full['DAYS_SINCE_LAST_ORDER'],
    'actual_churned': sample_pdf_full[label_column],
    'churn_probability': predictions,
    'prediction': (predictions > 0.5).astype(int)
})

print("\n🔮 Sample Predictions:")
print(results.to_string(index=False))
print(f"\n✅ Tested on {len(results)} sample customers")

## 10. Cleanup: Scale Down Cluster

In [ ]:
# Scale back to 1 node when done
print("Scaling cluster back to 1 node...")
scale_cluster(expected_cluster_size=1)
print(f"✅ Cluster scaled down to {len(get_nodes())} node")
print("\n🎉 Training complete! Model is ready for use in the Streamlit dashboard.")